On his channel, Ragusea says he posts recipe videos on Thursdays, so we will kill all videos not posted on Thursday

In [57]:
import pandas as pd
import datetime
import spacy
from quantulum3 import parser
sp = spacy.load('en_core_web_sm')
import nltk

In [38]:
rag_df = pd.read_csv("raw\AdamRaguseaVideos.csv")
rag_df.head()

,Unnamed: 0,Unnamed: 0.1,Video Name,Thumbnail URL,Video ID,length,description snippet,date published,transcript
0,0,0,Why dutched cocoa is different from natural cocoa,https://i.ytimg.com/vi/dT_LaNOhY4s/hqdefault.j...,dT_LaNOhY4s,"10 minutes, 2 seconds",Thanks to Audible for sponsoring this video! L...,2022-12-05 18:29:02.621087,what's the difference between natural cocoa a...
1,1,1,Do fiber supplements work? Do green powders wo...,https://i.ytimg.com/vi/RlUfyvtiE74/hqdefault.j...,RlUfyvtiE74,"50 minutes, 9 seconds",Thanks to Shaker & Spoon for sponsoring this e...,2022-12-03 18:29:02.621087,it's the Adam regusia podcast episode 35 powd...
2,2,2,Detroit-style pizza at home,https://i.ytimg.com/vi/ahxKAlbp6DU/hqdefault.j...,ahxKAlbp6DU,"8 minutes, 39 seconds",Thanks to Native for sponsoring this video! Sa...,2022-12-01 18:29:02.621087,traditional Detroit style pizza at home model...
3,3,3,AGE your raw egg eggnog,https://i.ytimg.com/vi/sflZWeCjdco/hqdefault.j...,sflZWeCjdco,"10 minutes, 14 seconds",Buy the Adam Ragusea chef knife for Christmas ...,2022-11-28 18:29:02.621087,eggnog like so many old-fashioned cocktails i...
4,4,4,Is it bad to stare at your phone while eating?...,https://i.ytimg.com/vi/L2w3gMM16tk/hqdefault.j...,L2w3gMM16tk,"53 minutes, 6 seconds",Thanks to Harry's for sponsoring this episode!...,2022-11-26 18:29:02.621087,it's the Adam regusia podcast Episode 34 past...


In [39]:
rag_df.drop(columns=rag_df.columns[0], 
        axis=1, 
        inplace=True)

In [40]:
rag_df.head()

,Unnamed: 0.1,Video Name,Thumbnail URL,Video ID,length,description snippet,date published,transcript
0,0,Why dutched cocoa is different from natural cocoa,https://i.ytimg.com/vi/dT_LaNOhY4s/hqdefault.j...,dT_LaNOhY4s,"10 minutes, 2 seconds",Thanks to Audible for sponsoring this video! L...,2022-12-05 18:29:02.621087,what's the difference between natural cocoa a...
1,1,Do fiber supplements work? Do green powders wo...,https://i.ytimg.com/vi/RlUfyvtiE74/hqdefault.j...,RlUfyvtiE74,"50 minutes, 9 seconds",Thanks to Shaker & Spoon for sponsoring this e...,2022-12-03 18:29:02.621087,it's the Adam regusia podcast episode 35 powd...
2,2,Detroit-style pizza at home,https://i.ytimg.com/vi/ahxKAlbp6DU/hqdefault.j...,ahxKAlbp6DU,"8 minutes, 39 seconds",Thanks to Native for sponsoring this video! Sa...,2022-12-01 18:29:02.621087,traditional Detroit style pizza at home model...
3,3,AGE your raw egg eggnog,https://i.ytimg.com/vi/sflZWeCjdco/hqdefault.j...,sflZWeCjdco,"10 minutes, 14 seconds",Buy the Adam Ragusea chef knife for Christmas ...,2022-11-28 18:29:02.621087,eggnog like so many old-fashioned cocktails i...
4,4,Is it bad to stare at your phone while eating?...,https://i.ytimg.com/vi/L2w3gMM16tk/hqdefault.j...,L2w3gMM16tk,"53 minutes, 6 seconds",Thanks to Harry's for sponsoring this episode!...,2022-11-26 18:29:02.621087,it's the Adam regusia podcast Episode 34 past...


In [41]:
searchfor = ['PODCAST', 'podcast', 'Podcast']
rag_df = rag_df[~rag_df['Video Name'].str.contains('|'.join(searchfor))]

In [29]:
def get_POS(text):
    sp_out = sp(text)
    sent_pos_arr = []
    for token in sp_out:
        sent_pos_arr.append(token.pos_)
    return sent_pos_arr

In [30]:
# rag_df['POS'] = rag_df['Video Name'].apply(get_POS)

In [31]:
# rag_df = rag_df[~rag_df['POS'].apply(lambda x: 'VERB' in x)]

In [43]:
rag_df = rag_df[~rag_df['Video Name'].apply(lambda x: 'Why' in x)]

In [82]:
def unit_thresh(text):
    words = text.split(' ')
    chunk_size = 10
    counter = 0
    for i in range(0, len(words), chunk_size):
        chunkedtext = words[i:i+chunk_size]
        chunkedstring = ' '.join(chunkedtext)
        # print(chunkedstring)
        try:
            units = parser.parse(chunkedstring)
            if units:
                for unit_meas in units:
                    if unit_meas.unit.name == 'dimensionless':
                        continue
                    else:
                        print(unit_meas.unit.name)
                        counter += 1
            if counter >= 3:
                return True
        except:
            continue
        
    return False

In [103]:
foodunits = ["teaspoon",
             "tablespoon",
             "fluid ounce",
             "gill",
             "cup",
             "pint",
             "quart",
             "gallon",
             "milliliter",
             "liter",
             "milligram",
             "gram",
             "kilogram",
             "pound",
             "ounce",
             "celcius",
             "farenheit",
             "inches",
             "centimeter",
             "meter",
             "feet",
             "millimeter",
             "cans",
             "bowl",
             "sprinkle",
             "pot",
             "minutes",
             "seconds"]

In [107]:
def check_food_unit(text):
    counter = 0
    try:
        text_arr = text.split(' ')
        for word in text_arr:
            if word in foodunits:
                counter+=1
            if counter > 5:
                return True
    except:
        return False
    return False

In [110]:
rag_df = rag_df[rag_df['transcript'].apply(check_food_unit)]

In [112]:
rag_df.info

<bound method DataFrame.info of      Unnamed: 0.1                                         Video Name  \
2               2                        Detroit-style pizza at home   
3               3                            AGE your raw egg eggnog   
7               7                 The SHOCKING SECRET to great chili   
10             10   Chicken and dumplings (biscuity, Southern-style)   
13             13     Cheesy broiled sheet pan pasta with vegetables   
..            ...                                                ...   
289           289  Shaved ice, snow cones, slushies, raspa, kakig...   
291           291                                     Salmon burgers   
295           295  Pancakes 201: Extra-fluffy, berry, chocolate a...   
297           297   Ice cream made with a pan, a spoon and a freezer   
299           299  Steak 101: How to choose a steak and cook it i...   

                                         Thumbnail URL     Video ID  \
2    https://i.ytimg.com/vi/ahxK

In [114]:
rag_df.to_csv("clean//AdamRaguseaVideosCleaned.csv", index = False)

In [118]:
rag_df = pd.read_csv("clean//AdamRaguseaVideosCleaned.csv")

In [119]:
rag_df = rag_df[~rag_df['Video Name'].apply(lambda x: 'How' in x)]

In [120]:
rag_df.head()

,Video Name,Thumbnail URL,Video ID,length,description snippet,date published,transcript
0,Detroit-style pizza at home,https://i.ytimg.com/vi/ahxKAlbp6DU/hqdefault.j...,ahxKAlbp6DU,"8 minutes, 39 seconds",Thanks to Native for sponsoring this video! Sa...,29:02.6,traditional Detroit style pizza at home model...
1,AGE your raw egg eggnog,https://i.ytimg.com/vi/sflZWeCjdco/hqdefault.j...,sflZWeCjdco,"10 minutes, 14 seconds",Buy the Adam Ragusea chef knife for Christmas ...,29:02.6,eggnog like so many old-fashioned cocktails i...
2,The SHOCKING SECRET to great chili,https://i.ytimg.com/vi/mb3k0wApWas/hqdefault.j...,mb3k0wApWas,"7 minutes, 52 seconds",Buy my custom chef knife before it sells out a...,29:02.6,Chile is a very broad category of stews or so...
3,"Chicken and dumplings (biscuity, Southern-style)",https://i.ytimg.com/vi/JgPfKZovZhA/hqdefault.j...,JgPfKZovZhA,"9 minutes, 6 seconds",Thanks to HelloFresh for sponsoring this video...,29:02.6,here's a slightly gussied up version of class...
4,Cheesy broiled sheet pan pasta with vegetables,https://i.ytimg.com/vi/XR0VKqfSHBk/hqdefault.j...,XR0VKqfSHBk,"7 minutes, 20 seconds",Thanks to Audible for sponsoring this video! L...,29:02.6,if you love baked pasta casseroles but only t...


In [121]:
rag_df.to_csv("clean//AdamRaguseaVideosCleaned.csv", index = False)